In [1]:
# Original class distribution
import matplotlib.pyplot as plt
import numpy as np
def plot_class_distribution(list_,labels,message = "Original Class Distribution"):
    unique, counts = np.unique(list_, return_counts=True)
    colors =['green','yellow']
    if len(unique) == 3 :
        colors.append('red')
        
    plt.bar(unique, counts, color=colors)
    plt.title(message)
    
    plt.xlabel('Class')
    plt.ylabel('Number of Samples')
    plt.show()
    
def plot_class_distribution_pie(list_, labels, message="Original Class Distribution"):
    unique, counts = np.unique(list_, return_counts=True)
    plt.pie(counts, labels=labels, colors=['green','yellow', 'red'], autopct='%1.1f%%')
    plt.title(message)
    plt.show()  

In [2]:
import pandas as pd
def read_csv_data(csv_file):
    data = pd.read_csv(csv_file)
    names = data['name'].tolist()
    labels = data['label'].tolist()
    texts = data['text'].tolist()
    return names, labels, texts

In [3]:
def count_uinque_labels(labels):
    unique = [0,0,0]
    for label in labels:
        if label.lower() =='approved':
            unique[0] += 1
        elif label.lower() == 'reject':
            unique[1] += 1
        else:
            unique[2] += 1        
    return {'approved':unique[0],'reject':unique[1],'pending':unique[2]}


In [4]:
csv_file = 'Dataset.csv'
names, labels, texts = read_csv_data(csv_file)

print(count_uinque_labels(labels))


{'approved': 198, 'reject': 8, 'pending': 30}


In [5]:
from Data import Data

def addToData1(names, labels, texts, data_obj, vectorization_technique):
    for name, label, text in zip(names, labels, texts):
        vector = vectorization_technique(text.strip()).numpy().tolist()
        data_obj.add_data(name.strip(), vector, label.strip())
    return data_obj

In [6]:
from USE import apply_USE 
obj = Data()
addToData1(names, labels, texts, obj, apply_USE)

In [7]:
X = obj.get_column(obj.col_document_vector)
y = obj.get_column(obj.col_decision)
print("Total number: ",len(X))
# set pending to reject
  
labels_count = count_uinque_labels(y)        
print(labels_count)


Total number:  236
{'approved': 198, 'reject': 8, 'pending': 30}


In [8]:
from imblearn.over_sampling import SMOTE
from collections import Counter

approvals = labels_count['approved']
rejects = labels_count['reject']
pending = labels_count['pending']
# class_proportions = 'auto'
upsample_class_proportions = {'Approved':approvals, 'Reject':rejects*2,'Pending':pending}
# upsample_class_proportions='auto'
smote = SMOTE(random_state=42, sampling_strategy=upsample_class_proportions)

X_resampled, y_resampled = smote.fit_resample(X, y)




In [9]:
# original_labels = y
# resampled_labels = y_resampled
# plot_class_distribution_pie(original_labels, labels=['approved', 'pending','rejected'], message="Original Class Distribution")
# plot_class_distribution_pie(resampled_labels, labels=['approved', 'pending','rejected'], message="Resampled Class Distribution")

# plot_class_distribution(original_labels, labels=['Class 0', 'Class 1'], message="Original Class Distribution")
# plot_class_distribution(resampled_labels, labels=['Class 0', 'Class 1'], message="Resampled Class Distribution")
print("Before Resampling: ",Counter(y))
print("After Resampling: ",Counter(y_resampled))



Before Resampling:  Counter({'Approved': 198, 'Pending': 30, 'Reject': 8})
After Resampling:  Counter({'Approved': 198, 'Pending': 30, 'Reject': 16})


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [11]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

    Approved       0.85      1.00      0.92        41
     Pending       1.00      0.14      0.25         7
      Reject       0.00      0.00      0.00         1

    accuracy                           0.86        49
   macro avg       0.62      0.38      0.39        49
weighted avg       0.86      0.86      0.81        49



c:\Users\Abdulrhman Fahmy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Abdulrhman Fahmy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Abdulrhman Fahmy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th

## auto 
```
Classification Report:
               precision    recall  f1-score   support

    Approved       0.94      0.78      0.85        40
     Pending       0.80      0.92      0.86        39
      Reject       0.98      1.00      0.99        40

    accuracy                           0.90       119
   macro avg       0.91      0.90      0.90       119
weighted avg       0.91      0.90      0.90       119

```





## approve *1 , reject *2 , pending *2:
```
Classification Report:
               precision    recall  f1-score   support

    Approved       0.80      0.97      0.88        36
     Pending       0.91      0.62      0.74        16
      Reject       0.00      0.00      0.00         3

    accuracy                           0.82        55
   macro avg       0.57      0.53      0.54        55
weighted avg       0.79      0.82      0.79        55

```
## approve *1 , reject *2 , pending:
```
Classification Report:
               precision    recall  f1-score   support

    Approved       0.85      1.00      0.92        41
     Pending       1.00      0.14      0.25         7
      Reject       0.00      0.00      0.00         1

    accuracy                           0.86        49
   macro avg       0.62      0.38      0.39        49
weighted avg       0.86      0.86      0.81        49
```






